In [ ]:
from huggingface_hub import login
login()

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are an assistant who always generate similar email to the one provided"},
    {"role": "user", "content": """Hi John,

Very happy to hear from you, could we have our meeting at your 12:30 or 1:00 pm on Tuesday - Friday?

Regards"""},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

def create_synthetic_emails(email_list:list):
    synthetic_emails = []
    for ith_email in email_list:
        messages = messages = [
            {"role": "system", "content": "You are an assistant who always generate similar email to the one provided"},
            {"role": "user", "content": f'{ith_email}'},
        ]
        outputs = pipeline(
            messages,
            max_new_tokens=256,
        )
        synthetic_emails.append(outputs[0]["generated_text"][-1])
    return synthetic_emails



In [14]:
text = "```json\n{\n  \"Subject\": \"Sports Roundup\",\n  \"Body\": \"## 🏀⚾🏈⚽ 🎾🏓🏸🏏🏒\n\nThis week in sports, there were some exciting developments across various disciplines. Here\'s a quick rundown:\n\n**🏀 NBA:** \n* The Golden State Warriors clinched their 8th NBA championship in their history, defeating the Boston Celtics in a thrilling six-game series. Stephen Curry was once again phenomenal, earning Finals MVP honors.\n\n**⚾ MLB:** \n* Shohei Ohtani continues to dominate both at the plate and on the mound. He recently hit his 30th home run of the season and is also among the league leaders in wins and ERA as a pitcher.\n* The New York Yankees are running away with the American League East, holding a comfortable lead over their rivals. Aaron Judge is on pace for a historic season, chasing Roger Maris\'s 61-homer record.\n\n**🏈 NFL:** \n* Preseason NFL games are underway, giving fans a glimpse of what to expect in the upcoming season. Several rookies have impressed so far, including quarterbacks Trevor Lawrence and Zach Wilson.\n* The reigning Super Bowl champion Los Angeles Rams are looking to repeat, but they face stiff competition from other contenders like the Kansas City Chiefs and Tampa Bay Buccaneers.\n\n**⚽ FIFA World Cup:** \n* With less than a year to go until the FIFA World Cup in Qatar, excitement is building across the globe. Several nations have already qualified, including host Qatar, Brazil, and Argentina.\n* Fans are eagerly anticipating the performance of star players like Lionel Messi, Cristiano Ronaldo, and Kylian Mbappé.\n## Other Sports News\n\n* In tennis, Novak Djokovic continues his reign at the top of the men\'s rankings. He recently won his 21st Grand Slam title at Wimbledon, tying the record held by Roger Federer and Rafael Nadal.\n* The Tokyo Olympics were a resounding success, showcasing incredible athletic performances across diverse sports.\n* Cricket fans around the world are looking forward to the upcoming ICC Men\'s T20 World Cup in Australia. India and England are among the favorites to win the tournament.\n\nLet me know if you\'d like more in-depth information on any specific sport or event.\"\n}\n```"

In [2]:
import json

In [13]:
import re

In [16]:
ans = re.split("`+json",text)
ans2 = re.split("`+$",ans[1])
ans3 = json.loads(ans2[0],strict=False)

In [ ]:
ans3

In [44]:

def get_json(text):
    error = ""
    problematic_text = ""
    try:
        answer = re.split("`+json",text)
        answer = re.split("`+$",answer[1])
        answer = json.loads(answer[0])
    except Exception as e:
        error = e
        problematic_text = text
        answer = json.loads(answer[0],strict=False)
    return answer,problematic_text,error



In [21]:
out = get_json(text)

In [ ]:
out[2]

In [28]:
with open("sample.json", "w") as outfile:
    json.dump(out[0], outfile)

In [29]:
with open('sample.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)

In [ ]:
json_object

In [31]:
emails = {ith_topic:[] for ith_topic in [1,2,3]}

In [ ]:
emails

In [ ]:
_ = load_dotenv(find_dotenv())

PROJECT_ID = os.environ["PROJECT_ID"]
REGION = os.environ["REGION"]
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession
import os 
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

PROJECT_ID = os.environ["PROJECT_ID"]
REGION = os.environ["REGION"]
vertexai.init(project=PROJECT_ID, location=REGION)

In [48]:


def get_chat_response(chat: ChatSession, prompt: str):
    response = chat.send_message(prompt)
    return response.text

def get_emails(topics,number_emails):

    model = GenerativeModel("gemini-1.0-pro")

    emails = {ith_topic:[] for ith_topic in topics}
    unparsed_emails = {ith_topic:[] for ith_topic in topics}
    wrong_format = {ith_topic:[] for ith_topic in topics}
    
    for i in range(number_emails):
        print(i)
        for ith_topic in topics:
            chat = model.start_chat()
            prompt = f"You're a helpful assistant. Write a short email in json format about {ith_topic}. Always use 'Subject' and 'Body' as the keys in the answer"
            ans_ = get_chat_response(chat, prompt)
            ans_, problematic_text, error = get_json(ans_)
            if len(problematic_text)==0:
                unparsed_emails[ith_topic].append({"text":problematic_text,"error":error})
            try:
                assert len(list(ans_.keys()))==2
                assert "Subject" in list(ans_.keys())
                assert "Body" in list(ans_.keys())
                emails[ith_topic].append(ans_)
            except Exception as e:
                wrong_format[ith_topic].append({"json":ans_,"error":e})
    return emails, unparsed_emails, wrong_format
        
    


In [ ]:
topics =["sports","vegetarian"]
ans = get_emails(topics,2)

In [ ]:
ans